<a href="https://colab.research.google.com/github/rsanchezgarc/AI-ML-analytics-IE/blob/main/notebooks/competitions/Winer_wine_competition_with_small_changes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install optuna lightgbm
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 2.0 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler
from dataclasses import dataclass
from typing import Tuple, List, Optional, Any, Dict

@dataclass
class ModelData:
    """Container for model data and metadata"""
    X_processed: np.ndarray
    X_test_processed: np.ndarray
    y: np.ndarray
    feature_names: List[str]
    classes: np.ndarray
    test_ids: np.ndarray

def is_gpu_available():
    """Check if GPU is available for XGBoost"""
    try:
        test_params = {
            'device': 'cuda:0',
        }
        test_model = xgb.XGBRegressor(**test_params)
        return True
    except Exception as e:
        print(f"GPU not available: {e}")
        return False

def label_encode_with_nan(series):
    """Label encode while preserving NaN values"""
    series = series.copy()
    nan_mask = series.isna()
    le = LabelEncoder()
    series[~nan_mask] = le.fit_transform(series[~nan_mask])
    series[nan_mask] = np.nan
    return series

def add_interactions(df):
    """Add domain-specific interaction terms"""
    df = df.copy()

    interactions = {
        ('fixed_acidity', 'volatile_acidity'): lambda x, y: x / y,
        ('residual_sugar', 'total_acidity'): lambda x, y: x / y,
        ('alcohol', 'total_acidity'): lambda x, y: x / y,
        ('density', 'alcohol'): lambda x, y: x / y
    }

    for (col1, col2), operation in interactions.items():
        if col1 in df.columns and col2 in df.columns:
            df[f'{col1}_{col2}_ratio'] = operation(df[col1], df[col2])

    return df

def preprocess_data(X: pd.DataFrame, X_test: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, List[str]]:
    """Preprocess features including interactions and encoding"""
    numerical_cols = X.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = ['Region'] if 'Region' in X else []

    X = add_interactions(X)
    X_test = add_interactions(X_test)

    numerical_cols = X.select_dtypes(include=np.number).columns.tolist()

    num_imputer = SimpleImputer(strategy='median')
    X_num = pd.DataFrame(num_imputer.fit_transform(X[numerical_cols]), columns=numerical_cols)
    X_test_num = pd.DataFrame(num_imputer.transform(X_test[numerical_cols]), columns=numerical_cols)

    if categorical_cols:
        for col in categorical_cols:
            X_num[col] = label_encode_with_nan(X[col])
            X_test_num[col] = label_encode_with_nan(X_test[col])

    return X_num.values, X_test_num.values, X_num.columns.tolist()

def get_model_params(trial: optuna.Trial, model_type: str, n_classes: int) -> Dict[str, Any]:
    """Get model-specific parameters based on trial suggestions"""
    if model_type == 'xgboost':
        params = {
            'objective': 'multi:softmax',
            'num_class': n_classes,
            'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 300, 1500),
            'max_depth': trial.suggest_int('max_depth', 4, 12),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0),
            'early_stopping_rounds': 50,
            'random_state': 42,
            'n_jobs': -1,
            'enable_categorical': True,
        }

        if is_gpu_available():
            params.update({
                'device': 'cuda:0',
            })
        else:
            params['tree_method'] = 'hist'

        return params
    else:  # Random Forest
        return {
            'n_estimators': trial.suggest_int('n_estimators', 300, 2500),
            'max_depth': trial.suggest_int('max_depth', 3, 30),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None]),
            'random_state': 42
        }

def compute_weights(class_weight_option: str, classes: np.ndarray, y: np.ndarray) -> np.ndarray:
    """Compute sample weights based on class weight option"""
    if class_weight_option:
        class_weights = compute_class_weight(class_weight_option, classes=classes, y=y)
        return np.array([class_weights[cls] for cls in y])
    return np.ones(len(y))

def train_model(model_type: str, params: Dict[str, Any], X_train: np.ndarray, y_train: np.ndarray,
                sample_weights: Optional[np.ndarray] = None, eval_set: Optional[List] = None,
                trial: Optional[optuna.Trial] = None) -> Any:
    """Train a model with given parameters and return it"""
    if model_type == 'xgboost':
        # if trial: # pruning_callback was interfering with early_stopping, that is why I disabled it here
        #     pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation_0-mlogloss')
            # params['callbacks'] = [pruning_callback]
        model = xgb.XGBClassifier(**params)
        fit_params = {
            'sample_weight': sample_weights,
            'eval_set': eval_set,
            'verbose': False
        }

        model.fit(X_train, y_train, **{k: v for k, v in fit_params.items() if v is not None})
    else:
        model = RandomForestClassifier(**params)
        model.fit(X_train, y_train)

    return model

def create_objective(data: ModelData):
    """Create and return the objective function for optimization"""
    def objective(trial: optuna.Trial) -> float:
        model_type = trial.suggest_categorical('model', ['xgboost', 'random_forest'])
        class_weight_option = trial.suggest_categorical('class_weight_option',
                                                      ['balanced',  None])

        sample_weights = compute_weights(class_weight_option, data.classes, data.y)
        f1_scores = []

        # cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=13)
        for train_idx, val_idx in cv.split(data.X_processed, data.y):
            X_train, X_val = data.X_processed[train_idx], data.X_processed[val_idx]
            y_train, y_val = data.y[train_idx], data.y[val_idx]

            params = get_model_params(trial, model_type, len(data.classes))
            if model_type != 'xgboost':
                params['class_weight'] = class_weight_option

            model = train_model(
                model_type,
                params,
                X_train,
                y_train,
                sample_weights[train_idx] if model_type == 'xgboost' else None,
                [(X_val, y_val)] if model_type == 'xgboost' else None,
                trial
            )

            preds = model.predict(X_val)
            f1_scores.append(f1_score(y_val, preds, average='macro'))

        mean_f1 = np.mean(f1_scores)
        trial.set_user_attr('f1_score', mean_f1)
        return mean_f1

    return objective

def load_and_preprocess_data() -> ModelData:
    """Load and preprocess the data"""
    train_data = pd.read_csv('train.csv')
    test_data = pd.read_csv('test.csv')

    target = 'quality'
    X = train_data.drop(columns=[target, 'ID'])
    y = train_data[target].values
    test_ids = test_data['ID'].values
    X_test = test_data.drop(columns=['ID'])

    X_processed, X_test_processed, feature_names = preprocess_data(X, X_test)
    classes = np.unique(y)

    return ModelData(
        X_processed=X_processed,
        X_test_processed=X_test_processed,
        y=y,
        feature_names=feature_names,
        classes=classes,
        test_ids=test_ids
    )

def main():
    """Main execution function"""
    gpu_available = is_gpu_available()
    print(f"GPU {'is' if gpu_available else 'is not'} available for training")

    # Load and prepare data
    data = load_and_preprocess_data()

    # Create and run study
    study = optuna.create_study(
        study_name="xgboost",
        direction='maximize',
        sampler=TPESampler(seed=42),
        pruner=optuna.pruners.MedianPruner(n_startup_trials=10),
        load_if_exists=True,
        storage='sqlite:///optuna_xgboost_study.db'
    )

    study.optimize(create_objective(data), n_trials=100, timeout=2*3600, show_progress_bar=True)

    # Train final model
    best_params = study.best_trial.params
    model_type = best_params.pop('model')
    class_weight_option = best_params.pop('class_weight_option')

    sample_weights = compute_weights(class_weight_option, data.classes, data.y)

    if model_type == 'xgboost':
        eval_set = [(data.X_processed, data.y)]
    else:
        best_params['class_weight'] = class_weight_option
        eval_set = None

    final_model = train_model(
        model_type,
        best_params,
        data.X_processed,
        data.y,
        sample_weights if model_type == 'xgboost' else None,
        eval_set
    )

    # Generate predictions
    test_preds = final_model.predict(data.X_test_processed)
    pd.DataFrame({'ID': data.test_ids, 'quality': test_preds}).to_csv('improved_submission.csv', index=False)
    print("Submission file created!")

if __name__ == "__main__":
    main()

[I 2025-02-04 09:56:33,685] Using an existing study with name 'xgboost' instead of creating a new one.


GPU is available for training


  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:56:34] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:56:34] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:56:37] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:56:37] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:56:40] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnin

[I 2025-02-04 09:56:51,715] Trial 32 finished with value: 0.5711502050074481 and parameters: {'model': 'xgboost', 'class_weight_option': 'balanced', 'learning_rate': 0.05845332860568505, 'n_estimators': 454, 'max_depth': 8, 'subsample': 0.7535341660233844, 'colsample_bytree': 0.7649083062977993, 'gamma': 0.003912978252459993, 'reg_alpha': 0.3496948661286037, 'reg_lambda': 0.0025132039125420388}. Best is trial 32 with value: 0.5711502050074481.


...

[I 2025-02-04 11:05:59,090] Trial 128 finished with value: 0.5803565188126025 and parameters: {'model': 'xgboost', 'class_weight_option': 'balanced', 'learning_rate': 0.015363161838340534, 'n_estimators': 982, 'max_depth': 8, 'subsample': 0.6908988493439076, 'colsample_bytree': 0.6717434136962608, 'gamma': 0.004241163506032611, 'reg_alpha': 0.9774768664961144, 'reg_lambda': 0.27647908502103136}. Best is trial 124 with value: 0.5879945658002083.
[I 2025-02-04 11:06:43,726] Trial 129 finished with value: 0.5345954522998818 and parameters: {'model': 'random_forest', 'class_weight_option': 'balanced', 'n_estimators': 1164, 'max_depth': 12, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt'}. Best is trial 124 with value: 0.5879945658002083.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:06:43] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:06:43] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:06:56] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:06:56] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:07:07] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnin

[I 2025-02-04 11:07:39,088] Trial 130 finished with value: 0.5737004660694629 and parameters: {'model': 'xgboost', 'class_weight_option': 'balanced', 'learning_rate': 0.019622793480313105, 'n_estimators': 1032, 'max_depth': 8, 'subsample': 0.7237808475489514, 'colsample_bytree': 0.6507663116342907, 'gamma': 0.049290546124193875, 'reg_alpha': 0.9520741805551396, 'reg_lambda': 0.3306352355383125}. Best is trial 124 with value: 0.5879945658002083.


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:07:39] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:07:39] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:07:54] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:07:54] WARNING: /workspace/src/context.cc:196: XGBoost is not compiled with CUDA support.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [11:08:11] WARNING: /workspace/src/context.cc:43: No visible GPU is found, setting device to CPU.
  warnin

[I 2025-02-04 11:08:56,598] Trial 131 finished with value: 0.5731155796964206 and parameters: {'model': 'xgboost', 'class_weight_option': 'balanced', 'learning_rate': 0.01336306699301117, 'n_estimators': 947, 'max_depth': 8, 'subsample': 0.7596655149899203, 'colsample_bytree': 0.8228878851343144, 'gamma': 0.01965448821309166, 'reg_alpha': 0.9165489944396574, 'reg_lambda': 0.3520607429404945}. Best is trial 124 with value: 0.5879945658002083.
Submission file created!
